In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


sns.set_theme(style="dark")


In [4]:
processed_data_path = '../data_processed/'
master_file_path = os.path.join(processed_data_path, 'indicadores_mensal.csv')


df = pd.read_csv(master_file_path, index_col='data', parse_dates=['data'])

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 67 entries, 2020-01-31 to 2025-07-31
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   selic_media_mensal        67 non-null     float64
 1   pmc_vendas_varejo         67 non-null     float64
 2   gtrends_magazine_luiza    67 non-null     float64
 3   gtrends_casas_bahia       67 non-null     float64
 4   gtrends_mercado_livre     67 non-null     float64
 5   ipca_variacao_mensal      67 non-null     float64
 6   icc_confianca_consumidor  67 non-null     float64
 7   desemprego_taxa           67 non-null     float64
dtypes: float64(8)
memory usage: 4.7 KB
